# KA SEIRSPlus
> backtesting on KA data

- toc: true 
- badges: false
- comments: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [1]:
# hide
# !pip install seirsplus
# !pip install -U plotly
# !pip install fuzzywuzzy
import contextlib
import io
import json
import random
import sys
import warnings
from pathlib import Path
from typing import List, Union
from urllib.request import urlopen

import pandas as pd
import plotly.graph_objects as go
from branca.colormap import linear
from fuzzywuzzy import process
from IPython.display import Latex, Markdown, display
from IPython.utils import io
from ipywidgets import (
    HTML,
    FloatLogSlider,
    FloatSlider,
    GridBox,
    HBox,
    IntSlider,
    Label,
    Layout,
    Output,
    SelectionSlider,
    VBox,
    interactive,
)
from seirsplus.models import *
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning:

Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning



In [2]:
# hide
ka_df = pd.read_csv("KarnatakaDistrictInfo.csv")
ka_df["Population"] = pd.to_numeric(
    ka_df["Population"].apply(lambda x: x.replace(",", ""))
)
ka_df

,District,Population,Inflow
0,Bagalkot,1889752,26
1,Bangalore,8612474,7315
2,Belgaum,4779661,16
3,Bellary,2452595,44
4,Bidar,1703300,0
5,Bijapur,2177331,0
6,Chamarajanagar,1020791,6
7,Chikkaballapura,1255104,9
8,Chikmagalur,1137961,27
9,Chitradurga,1659456,0


In [0]:
# hide
with urlopen(
    "https://raw.githubusercontent.com/covid19india/api/master/state_district_wise.json"
) as response:
    district = json.load(response)

In [4]:
district_maps = {}
for dis in list(district["Karnataka"]["districtData"].keys()):
    sel = process.extractOne(dis, ka_df["District"])
    if dis == "Kalaburagi":
        district_maps["Gulbarga"] = dis
    else:
        district_maps[sel[0]] = dis

district_maps

{'Bagalkot': 'Bagalkote',
 'Bangalore': 'Bengaluru',
 'Bangalore Rural': 'Bengaluru Rural',
 'Belgaum': 'Belagavi',
 'Bellary': 'Ballari',
 'Bidar': 'Bidar',
 'Chikkaballapura': 'Chikkaballapura',
 'Chitradurga': 'Chitradurga',
 'Dakshina Kannada': 'Dakshina Kannada',
 'Davanagere': 'Davanagere',
 'Dharwad': 'Dharwad',
 'Gadag': 'Gadag',
 'Gulbarga': 'Kalaburagi',
 'Kodagu': 'Kodagu',
 'Mandya': 'Mandya',
 'Mysore': 'Mysuru',
 'Tumkur': 'Tumakuru',
 'Udupi': 'Udupi',
 'Uttara Kannada': 'Uttara Kannada'}

In [5]:
ka_df["CaseCounts"] = ka_df["District"].apply(
    lambda x: district["Karnataka"]["districtData"][district_maps[x]]["confirmed"]
    if x in district_maps.keys()
    else 0
)
ka_df

,District,Population,Inflow,CaseCounts
0,Bagalkot,1889752,26,5
1,Bangalore,8612474,7315,58
2,Belgaum,4779661,16,7
3,Bellary,2452595,44,6
4,Bidar,1703300,0,10
5,Bijapur,2177331,0,0
6,Chamarajanagar,1020791,6,0
7,Chikkaballapura,1255104,9,10
8,Chikmagalur,1137961,27,0
9,Chitradurga,1659456,0,1


In [0]:
# hide
def get_infections(
    initI: int = 100,
    initN: int = 10 ** 5,
    days_N: int = 21,
    beta: float = 2.4,
    sigma=1 / 5.2,
    gamma=1 / 12.39,
    beta_D=0.000,
    mu_D=0.02,
    theta_E=0.0002,
    theta_I=0.002,
    psi_E=0.2,
    psi_I=1.0,
) -> List[int]:
    model = SEIRSModel(
        beta=beta,
        sigma=sigma,
        gamma=gamma,
        initN=initN,
        initI=initI,
        beta_D=beta_D,
        mu_D=mu_D,
        theta_E=theta_E,
        theta_I=theta_I,
        psi_E=psi_E,
        psi_I=psi_I,
    )
    with io.capture_output() as captured:
        model.run(T=days_N)
    S = model.numS  # time series of S counts
    E = model.numE  # time series of E counts
    I = model.numI  # time series of I counts
    D_E = model.numD_E  # time series of D_E counts
    D_I = model.numD_I  # time series of D_I counts
    R = model.numR  # time series of R counts
    F = model.numF  # time series of F counts
    t = model.tseries  # time values corresponding to the above time series
    return {"detected_exposed": D_E, "detected_infected": D_I, "model": model, "t": t}

In [0]:
# hide
def get_risk_estimates(
    state_estimated_df: pd.DataFrame,
    beta: float,
    sigma=1 / 5.2,
    gamma=1 / 12.39,
    beta_D=0.000,
    mu_D=0.02,
    theta_E=0.0002,
    theta_I=0.002,
    psi_E=0.2,
    psi_I=1.0,
) -> Union[List, List]:
    days_N = 21
    atrisk_day14, atrisk_day21 = [], []
    for row in state_estimated_df[["initI", "initN", "District"]].iterrows():
        initI, initN, district = row[1][0], int(row[1][1]), row[1][2]
        #     print(type(initI), type(initN))
        infection_results = get_infections(
            initI=initI,
            initN=initN,
            days_N=days_N,
            beta=beta,
            sigma=sigma,
            gamma=gamma,
            beta_D=beta_D,
            mu_D=mu_D,
            theta_E=theta_E,
            theta_I=theta_I,
            psi_E=psi_E,
            psi_I=psi_I,
        )
        detected_infected = infection_results["detected_infected"]
        day14 = int(14 * len(detected_infected) / days_N)
        case_count_day14 = int(infection_results["detected_infected"][day14])
        case_count_day21 = int(infection_results["detected_infected"][-1])
        atrisk_day14.append(case_count_day14)
        atrisk_day21.append(case_count_day21)
    return atrisk_day14, atrisk_day21, infection_results

In [0]:
# hide_input
# @title Assumptions
percent_travellers_infected = 0.039  # @param {type:"slider", min:0, max:1e-1, step:0.001}
unknown_to_known_travelers = 23  # @param {type:"slider", min:5, max:50, step:1}
beta = 0.6  # @param {type:"slider", min:0.5, max:5.0, step:0.1}

# display(Markdown(f"#Assumptions \npercent_travellers_infected :{percent_travellers_infected}\nunknown_to_known_travelers={unknown_to_known_travelers}\nbeta:{beta} "))

In [9]:
import math
math.ceil(0.2)

1

In [0]:
# hide
def estimate(
    percent_travellers_infected: float,
    state_df: pd.DataFrame,
    beta: float,
    bus_to_passenger=0.01, 
    public_bus_to_private_bus=0.01,
    percent_qtn_infected=0.01,
    sigma=1 / 5.2,
    gamma=1 / 12.39,
    beta_D=0.000,
    mu_D=0.02,
    theta_E=0.0002,
    theta_I=0.002,
    psi_E=0.2,
    psi_I=1.0,
):

    state_estimated_df = state_df.copy()
    passengers_infected = public_bus_to_private_bus * bus_to_passenger * percent_travellers_infected
    state_estimated_df["initI"] = (
        passengers_infected + percent_qtn_infected * state_df["Inflow"]
    ).astype(int)
    state_estimated_df["initN"] = state_estimated_df["Population"].astype(int)

    (
        state_estimated_df[f"day14"],
        state_estimated_df[f"day21"],
        infection_results,
    ) = get_risk_estimates(
        state_estimated_df,
        beta=beta,
        sigma=sigma,
        gamma=gamma,
        beta_D=beta_D,
        mu_D=mu_D,
        theta_E=theta_E,
        theta_I=theta_I,
        psi_E=psi_E,
        psi_I=psi_I,
    )
    return state_estimated_df

In [33]:
# hide
grid = {
    # "sigma": [1 / (i / 10) for i in range(50, 140, 10)],
    # "gamma": [1 / (i / 10) for i in range(100, 200, 10)],
    "beta_D": [i / 1000 for i in range(1, 6)],
    "percent_qtn_infected":[i / 1000 for i in range(1, 100, 10)],
    "public_bus_to_private_bus":[i for i in range(30, 50)],
    "bus_to_passenger":[i for i in range(50, 80, 5)],
    "percent_travellers_infected":[i / 1000 for i in range(1, 100, 10)],
    "mu_D": [i / 100 for i in range(1, 6)],
    "theta_E": [i / 10000 for i in range(1, 6)],
    # "theta_I": [i / 1000 for i in range(1, 6)],
    # "psi_E": [i / 10 for i in range(5)],
    # "psi_I": [i / 10 for i in range(5)],
    "beta": [i / 10 for i in range(5, 9)],
}
grid

{'beta': [0.5, 0.6, 0.7, 0.8],
 'beta_D': [0.001, 0.002, 0.003, 0.004, 0.005],
 'bus_to_passenger': [50, 55, 60, 65, 70, 75],
 'mu_D': [0.01, 0.02, 0.03, 0.04, 0.05],
 'percent_qtn_infected': [0.001,
  0.011,
  0.021,
  0.031,
  0.041,
  0.051,
  0.061,
  0.071,
  0.081,
  0.091],
 'percent_travellers_infected': [0.001,
  0.011,
  0.021,
  0.031,
  0.041,
  0.051,
  0.061,
  0.071,
  0.081,
  0.091],
 'public_bus_to_private_bus': [30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49],
 'theta_E': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [34]:
# hide
len(list(ParameterGrid(grid)))

6000000

In [21]:
ka_df[ka_df.Inflow != 0][ka_df.District !="Bangalore"][ka_df.District !="Bangalore Rural"]

,District,Population,Inflow,CaseCounts
0,Bagalkot,1889752,26,5
2,Belgaum,4779661,16,7
3,Bellary,2452595,44,6
6,Chamarajanagar,1020791,6,0
7,Chikkaballapura,1255104,9,10
8,Chikmagalur,1137961,27,0
10,Dakshina Kannada,2089649,756,10
11,Davanagere,1945497,58,2
12,Dharwad,1847023,39,1
13,Gadag,1064570,16,1


In [36]:
# hide
config = {}


def megs_shitty_grid_search(grid):
    prev = 1000000
    for combination in tqdm(random.sample(list(ParameterGrid(grid)), 1400)):
        ka_estimated_df = estimate(
                percent_travellers_infected= combination["percent_travellers_infected"],
                state_df= ka_df[ka_df.Inflow != 0][ka_df.District !="Bangalore"][ka_df.District !="Bangalore Rural"],
                beta = combination["beta"],
                bus_to_passenger=combination["bus_to_passenger"], 
                public_bus_to_private_bus=combination["public_bus_to_private_bus"],
                percent_qtn_infected=combination["percent_qtn_infected"],
                mu_D=combination["mu_D"], 
                theta_E=combination["theta_E"]
        )
        error = mean_absolute_error(
            ka_estimated_df["day14"], ka_estimated_df["CaseCounts"]
        )
        if error < prev:
            prev = error
            config = combination
            print(f"error : {error}")
    return config


config = megs_shitty_grid_search(grid)
config

  0%|          | 2/1400 [00:00<02:41,  8.64it/s]

error : 4.8


  1%|          | 8/1400 [00:00<02:35,  8.96it/s]

error : 3.84


  2%|▏         | 27/1400 [00:03<02:30,  9.15it/s]

...

error : 3.56


 70%|███████   | 986/1400 [01:57<00:46,  8.84it/s]

error : 3.52


100%|██████████| 1400/1400 [02:43<00:00,  8.55it/s]


{'beta': 0.7,
 'beta_D': 0.001,
 'bus_to_passenger': 50,
 'mu_D': 0.01,
 'percent_qtn_infected': 0.081,
 'percent_travellers_infected': 0.011,
 'public_bus_to_private_bus': 39,
 'theta_E': 0.0004}

In [37]:
# hide
config 

{'beta': 0.7,
 'beta_D': 0.001,
 'bus_to_passenger': 50,
 'mu_D': 0.01,
 'percent_qtn_infected': 0.081,
 'percent_travellers_infected': 0.011,
 'public_bus_to_private_bus': 39,
 'theta_E': 0.0004}

In [38]:
# hide
ka_estimated_df = estimate(
                percent_travellers_infected= config["percent_travellers_infected"],
                state_df= ka_df,
                beta = config["beta"],
                bus_to_passenger=config["bus_to_passenger"], 
                public_bus_to_private_bus=config["public_bus_to_private_bus"],
                percent_qtn_infected=config["percent_qtn_infected"],
                mu_D=config["mu_D"], 
                theta_E=config["theta_E"]
        )
ka_estimated_df[["District", "CaseCounts", "day14"]]

,District,CaseCounts,day14
0,Bagalkot,5,2
1,Bangalore,58,56
2,Belgaum,7,2
3,Bellary,6,2
4,Bidar,10,1
5,Bijapur,0,1
6,Chamarajanagar,0,1
7,Chikkaballapura,10,2
8,Chikmagalur,0,2
9,Chitradurga,1,1


In [0]:
# hide
ka_estimated_df["Day 21 Infections"] = ka_estimated_df["day21"]
ka_estimated_df["Day 14 Infections"] = ka_estimated_df["day14"]

In [0]:
# hide
ka_estimated_df["text"] = (
    "District : "
    + ka_estimated_df["District"].astype(str)
    + "<br>"
    + "Population: "
    + ka_estimated_df["Population"].astype(str)
    + "<br>"
    + "Day 14 Infections : "
    + ka_estimated_df["Day 14 Infections"].astype(str)
    + "<br>"
    + "Day 21 Infections : "
    + ka_estimated_df["Day 21 Infections"].astype(str)
    + "<br>"
    + "Travellers: "
    + ka_estimated_df["Inflow"].astype(str)
)

ka_estimated_df.head()

# District vs AtRisk

In [0]:
# hide_input
ka_estimated_df[["District", "Day 14 Infections", "Day 21 Infections", "CaseCounts"]]

# Map Plot

In [0]:
# hide
with urlopen(
    "https://raw.githubusercontent.com/meghanabhange/CovidSeer/backtesting/_notebooks/karnataka.json"
) as response:
    KA = json.load(response)

In [0]:
# hide
fig = go.Figure(
    data=go.Choropleth(
        locations=ka_estimated_df["District"],
        z=ka_estimated_df["day21"].astype(float),
        colorscale="Blues",
        autocolorscale=False,
        reversescale=False,
        marker_line_color="darkgray",
        marker_line_width=0.5,
        geojson=KA,
        featureidkey="properties.district",
        colorbar_title="At Risk",
        text=ka_estimated_df["text"],
        hoverinfo="text",
    )
)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text="District Analysis Karnataka: Map")

In [0]:
# hide_input
fig.show()